In [1]:

from bloqade import rydberg_h, piecewise_linear,get_capabilities,  piecewise_constant, waveform, cast
from bloqade.atom_arrangement import ListOfLocations, Lieb, Square, Chain, Honeycomb, Kagome, Triangular, Rectangular
from bokeh.io import output_notebook # to plot "show()" on the notebook, without opening a browser
import networkx as nx
import os
import matplotlib.pyplot as plt
import numpy as np
output_notebook()

import networkx as nx
import os
import matplotlib.pyplot as plt
import numpy as np

from bokeh.io import output_notebook, output_file # to plot "show()" on the notebook, without opening a browser
from bokeh.io import export_svg, export_svgs, show, curdoc
from bokeh.models import Plot
from bokeh.io.export import get_screenshot_as_png


def testF(event):
    print("Aaaa", event)


def testCurdoc():
    doc = curdoc()
    doc.on_change(testF)

    plots = list(doc.select({"type": Plot}))
    print("Ho trovato", len(plots), "plot nel documento Bokeh")
    for i, p in enumerate(plots):
        p.output_backend = "svg"
        export_svgs(p, filename=f"bokeh_plot_{i}.svg")



def getMHz(Rb, C6 = float(get_capabilities().capabilities.rydberg.c6_coefficient)):
    return C6 / (Rb ** 6)


Rb = 9.5
omega_MHz_value = getMHz(Rb)

omega_MHz = [0.0, omega_MHz_value, omega_MHz_value,omega_MHz_value,omega_MHz_value, 0.0]
delta_MHz = [-omega_MHz_value, -omega_MHz_value,0 ,0,omega_MHz_value, omega_MHz_value]
durations = [ 0.25,1,0.1,2.4, 0.25]
omega_MHz_value = getMHz(Rb = 9.5)
Delta = piecewise_linear(durations, [x  for x in delta_MHz])
Omega = piecewise_linear(durations, [x for x in omega_MHz])
durations = [0.25, 1, 0.1, 2.4, 0.25]
#atom_coordinate_1 = [(9, 0.0),(0,9) ,(9, 9), (18, 0), (18, 9), ((4.5+9), 12), ((18)+9, 9)]
atom_coordinate_1 = [(9, 0.0),(0,9) ,(9, 9), (18, 0), (18, 9), ((18)+9, 9)] # Molti MIS
#atom_coordinate_1 = [(0, 0),(6, 0),(12, 0),(18, 0),(9, 6),(15, 6)]# singolo MIS 

omega_MHz = [0.0, omega_MHz_value, omega_MHz_value,omega_MHz_value,omega_MHz_value, 0.0]
delta_MHz = [-omega_MHz_value, -omega_MHz_value,0 ,0,omega_MHz_value, omega_MHz_value]
# Creazione del programma
program = rydberg_h(atom_coordinate_1, detuning=Delta, amplitude=Omega, phase=None)


reg = program.parse_register() #reg.show()
seq = program.parse_sequence() #seq.show()
output = program.bloqade.python().run(shots=1000, interaction_picture=True)
outputReport = output.report() #outputReport.show()

#REGISTRO
fig_reg = reg.figure()
fig_reg.output_backend = "svg"
export_svg(fig_reg, filename="register.svg")

show(fig_reg)
######SEQUENZA
seq = program.parse_sequence()
tabs = seq.figure()   # Tabs con i grafici della sequence
# prende tutti i Plot dentro ai Tabs e li converte in lista
plots = list(tabs.select({"type": Plot}))

print("Ho trovato", len(plots), "plot nella sequence")

# esporta ogni plot come SVG separato
for i, p in enumerate(plots):
    p.output_backend = "svg"                 # fondamentale per SVG
    export_svgs(p, filename=f"sequence_{i}.svg")

# continua a mostrare i Tabs interattivi
show(tabs)
output.report().show()


Loading BokehJS ...

Ho trovato 2 plot nella sequence


0.0 27000000.0 0.0 9000000.0


In [2]:
from bokeh.models import Plot

FONT = "DejaVu Sans"  # o "Helvetica"

###### SEQUENZA
seq = program.parse_sequence()
tabs = seq.figure()

plots = list(tabs.select({"type": Plot}))
print("Ho trovato", len(plots), "plot nella sequence")

ylabels = ["Delta(t) (rad/s)", "Omega(t) (rad/s)"]

SVG_WIDTH  = 350
SVG_HEIGHT = 250

for i, p in enumerate(plots):
    for ax in p.xaxis:
        ax.axis_label = "Time (μs)"
        ax.axis_label_text_font = FONT
        ax.major_label_text_font = FONT

    for ax in p.yaxis:
        ax.axis_label = ylabels[i]
        ax.axis_label_text_font = FONT
        ax.major_label_text_font = FONT

    for ax in list(p.xaxis) + list(p.yaxis):
        ax.axis_label_text_font_size = "10pt"
        ax.axis_label_text_font_style = "normal"
        ax.axis_label_standoff = 10

    p.width = SVG_WIDTH
    p.height = SVG_HEIGHT

    p.output_backend = "svg"
    export_svgs(p, filename=f"sequence_{i}.svg",
                width=SVG_WIDTH, height=SVG_HEIGHT)

show(tabs)

Ho trovato 2 plot nella sequence


In [13]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import Viridis256
from bokeh.layouts import row

# ====================== DATI DI BASE (una volta sola) ======================

report = output.report()

counts_list = report.counts()
if len(counts_list) == 0:
    raise ValueError("Nessun task nel report!")

counts0 = counts_list[0]   # primo task: OrderedDict(bitstring -> conteggio)

# lista completa di bitstring e conteggi, in ordine di frequenza decrescente
items = list(counts0.items())
all_states  = [s for (s, c) in items]
all_counts  = [c for (s, c) in items]

# prendiamo solo le prime 10 colonne
TOP_K = 15
top_states = all_states[:TOP_K]
top_counts = all_counts[:TOP_K]

# coordinate degli atomi
xs = [x for (x, y) in atom_coordinate_1]
ys = [y for (x, y) in atom_coordinate_1]
def make_plots_for_state(k, base_name=None, show_in_notebook=True):
    """
    k: indice (0–9) della colonna che vuoi 'selezionare'
       all'interno delle prime TOP_K colonne.
    base_name: prefisso per i nomi dei file svg.
    """
    if k < 0 or k >= len(top_states):
        raise IndexError(f"k fuori range: 0 <= k < {len(top_states)}")

    state = top_states[k]
    count = top_counts[k]

    # --------------------- Istogramma con UNA sola barra evidenziata (prime 10) ---------------------
    # etichette x: [0], [1], [2], ...
    x_labels = [f"[{i}]" for i in range(len(top_states))]

    bar_colors = [
        "#d95f02" if i == k else "#fdd0a2"
        for i in range(len(top_states))
    ]

    source_hist = ColumnDataSource(dict(
        x_labels = x_labels,
        counts   = top_counts,
        color    = bar_colors,
    ))

    fig_hist = figure(
        x_range=x_labels,          # usa le etichette [0], [1], ...
        width=400,
        height=500,
        x_axis_label="assignment index",
        y_axis_label="Counts",
        toolbar_location=None,
    )

    fig_hist.vbar(
        x="x_labels",
        top="counts",
        width=0.9,
        source=source_hist,
        fill_color="color",
        line_color="color",
    )

    fig_hist.xaxis.major_label_orientation = np.pi / 2

    # --------------------- Rydberg density per SOLO questo stato ---------------------
    bits = np.array(list(state), dtype=int)
    rydberg_density = 1 - bits          # prob(rydberg) = 1 - bit

    source_density = ColumnDataSource(dict(
        x       = xs,
        y       = ys,
        density = rydberg_density,
    ))

    color_mapper = LinearColorMapper(palette=Viridis256, low=0.0, high=1.0)

    fig_density = figure(
        width=400,
        height=400,
        match_aspect=True,
        title=f"Rydberg density (stato {state}, colonna {k})",
    )

    fig_density.circle(
        x="x",
        y="y",
        size=40,
        source=source_density,
        fill_color={"field": "density", "transform": color_mapper},
        line_color="black",
    )

    color_bar = ColorBar(
        color_mapper=color_mapper,
        label_standoff=8,
        location=(0, 0),
    )
    fig_density.add_layout(color_bar, "right")

    # --------------------- Export SVG ---------------------
    if base_name is None:
        base_name = f"state_{k}"

    fig_hist.output_backend = "svg"
    export_svgs(fig_hist, filename=f"{base_name}_hist.svg")

    fig_density.output_backend = "svg"
    export_svgs(fig_density, filename=f"{base_name}_rydberg_density.svg")

    if show_in_notebook:
        show(row(fig_hist, fig_density))
make_plots_for_state(3, base_name="colonna_0")

In [14]:
from bokeh.models import ColumnDataSource, PrintfTickFormatter, FixedTicker

# stato singolo: per esempio la seconda colonna (indice 1)
state0 = all_states[3]
bits0 = np.array(list(state0), dtype=int)

# ========== SCALING per avere i tick 0, 1e7, 2e7, 3e7 ==========
# portiamo il massimo delle coordinate x a 3e7
max_x = max(xs)
scale = 3e7 / max_x

xs_scaled = [x * scale for x in xs]
ys_scaled = [y * scale for y in ys]

source_regstate = ColumnDataSource(dict(
    x     = xs_scaled,
    y     = ys_scaled,
    color = ["#000000" if b == 0 else "#fbfcbf" for b in bits0],
))

fig_regstate = figure(
    width=500,
    height=500,
    match_aspect=True,
    x_range=(-1e7, 4e7),
    y_range=(-1e7, 4e7),
    toolbar_location=None,
)

# palline come nello screenshot
fig_regstate.circle(
    x="x",
    y="y",
    size=60,
    source=source_regstate,
    fill_color="color",
    line_color="#000000",
    line_width=2,
)

# ========== ASSI, LABEL, TICK COME BLOQADE ==========

# label asse x in corsivo
fig_regstate.xaxis.axis_label = "(um)"
fig_regstate.xaxis.axis_label_text_font_style = "italic"

# tick fissi 0, 1e7, 2e7, 3e7
ticks = [0, 1e7, 2e7, 3e7]
fig_regstate.xaxis.ticker = FixedTicker(ticks=ticks)
fig_regstate.yaxis.ticker = FixedTicker(ticks=ticks)

# formato scientifico tipo 0.000e+00, 1.000e+07, ...
fig_regstate.xaxis.formatter = PrintfTickFormatter(format="%0.3e")
fig_regstate.yaxis.formatter = PrintfTickFormatter(format="%0.3e")

# griglia leggera come quella che vedi nel plot originale
fig_regstate.xgrid.grid_line_alpha = 1
fig_regstate.ygrid.grid_line_alpha = 1

# ========== EXPORT SVG ==========

fig_regstate.output_backend = "svg"
export_svgs(fig_regstate, filename="report_reg_state_state0.svg")
show(fig_regstate)


In [15]:
import xml.etree.ElementTree as ET
import re

def _parse_size(value):
    """
    Restituisce (numero, unità) da stringhe tipo '400pt', '500', '300px'.
    """
    if value is None:
        return 0.0, ""
    m = re.match(r"([0-9.]+)([a-zA-Z%]*)", value)
    if not m:
        return 0.0, ""
    return float(m.group(1)), m.group(2)

def combine_svgs(svg_left, svg_right, svg_out, gap=20):
    """
    svg_left  : path dell'SVG dell'istogramma (sinistra)
    svg_right : path dell'SVG del report_reg_state_state (destra)
    svg_out   : path dell'SVG combinato di output
    gap       : spazio orizzontale tra le due figure (stessa unità dell'SVG)
    """
    # carico i due svg
    tree_left  = ET.parse(svg_left)
    root_left  = tree_left.getroot()
    tree_right = ET.parse(svg_right)
    root_right = tree_right.getroot()

    # prendo larghezze/altezze e unità
    wL, unit_wL = _parse_size(root_left.get("width"))
    hL, unit_hL = _parse_size(root_left.get("height"))
    wR, _       = _parse_size(root_right.get("width"))
    hR, _       = _parse_size(root_right.get("height"))

    total_w = wL + gap + wR
    total_h = max(hL, hR)

    # aggiorno dimensioni e viewBox del root sinistro (che diventa il root combinato)
    unit = unit_wL or "pt"
    root_left.set("width",  f"{total_w}{unit}")
    root_left.set("height", f"{total_h}{unit}")
    root_left.set("viewBox", f"0 0 {total_w} {total_h}")

    # namespace SVG
    svg_ns  = root_left.tag.split('}')[0].strip('{')
    g_tag   = f"{{{svg_ns}}}g"
    defs_tag = f"{{{svg_ns}}}defs"

    # unisco eventuali <defs>
    defs_left  = root_left.find(defs_tag)
    defs_right = root_right.find(defs_tag)
    if defs_right is not None:
        if defs_left is None:
            defs_left = ET.SubElement(root_left, defs_tag)
        for child in list(defs_right):
            defs_left.append(child)

    # creo un gruppo traslato per la figura di destra
    g_right = ET.Element(g_tag, attrib={"transform": f"translate({wL + gap},0)"})
    for child in list(root_right):
        if child.tag == defs_tag:
            continue  # defs già uniti
        g_right.append(child)

    # aggiungo il gruppo a destra dentro l'SVG combinato
    root_left.append(g_right)

    # salvo l'SVG finale
    tree_left.write(svg_out, encoding="utf-8", xml_declaration=True)
    print(f"SVG combinato salvato in: {svg_out}")
    

combine_svgs(
    "colonna_0_hist.svg",
    "report_reg_state_state0.svg",
    "colonna_0_hist+regstate.svg"
)

SVG combinato salvato in: colonna_0_hist+regstate.svg


In [17]:
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import networkx as nx

# Coordinate dei nodi
coordinates_list = [(9,0), (0,9), (9,9), (18,0), (18,9), (27,9)]
coordinates = {str(i): coordinates_list[i] for i in range(len(coordinates_list))}

Rb = 9.5  # raggio del disco di connessione
margin = 13.5

# Costruisco un grafo a disco (unit disk graph generalizzato con raggio Rb)
G = nx.Graph()
G.add_nodes_from(coordinates.keys())

def dist(p, q):
    return math.hypot(p[0]-q[0], p[1]-q[1])

nodes = list(coordinates.keys())
for i in range(len(nodes)):
    for j in range(i+1, len(nodes)):
        u, v = nodes[i], nodes[j]
        if dist(coordinates[u], coordinates[v]) <= Rb:
            G.add_edge(u, v)

# Figura + assi
fig, ax = plt.subplots(figsize=(margin, margin))
ax.set_aspect('equal', adjustable='box')
ax.set_facecolor((1., 1., 1., 0.))  # trasparente

# Disegno il grafo
nx.draw_networkx(
    G,
    pos=coordinates,
    with_labels=True,
    font_size=20,
    node_size=2000,
    ax=ax
)

# Cerchi di raggio Rb attorno a ciascun nodo
for center in coordinates.values():
    ax.add_patch(Circle(center, Rb, alpha=0.1))

# Limiti del grafico
xs = [p[0] for p in coordinates.values()]
ys = [p[1] for p in coordinates.values()]
ax.set_xlim(min(xs)-margin, max(xs)+margin)
ax.set_ylim(min(ys)-margin, max(ys)+margin)
ax.axis('off')

# Salvataggio con sfondo trasparente
plt.savefig("outputFigure.svg", transparent=True, bbox_inches='tight')
plt.close(fig)